# Inference Pipeline with Scikit-learn and Linear Learner
Typically a Machine Learning (ML) process consists of few steps: data gathering with various ETL jobs, pre-processing the data, featurizing the dataset by incorporating standard techniques or prior knowledge, and finally training an ML model using an algorithm. 
In many cases, when the trained model is used for processing real time or batch prediction requests, the model receives data in a format which needs to pre-processed (e.g. featurized) before it can be passed to the algorithm. In the following notebook, we will demonstrate how you can build your ML Pipeline leveraging the Sagemaker Scikit-learn container and SageMaker Linear Learner algorithm & after the model is trained, deploy the Pipeline (Data preprocessing and Lineara Learner) as an Inference Pipeline behind a single Endpoint for real time inference and for batch inferences using Amazon SageMaker Batch Transform.

We will demonstrate this using the Abalone Dataset to guess the age of Abalone with physical features. The dataset is available from [UCI Machine Learning](https://archive.ics.uci.edu/ml/datasets/abalone); the aim for this task is to determine age of an Abalone (a kind of shellfish) from its physical measurements. We'll use Sagemaker's Scikit-learn container to featurize the dataset so that it can be used for training with Linear Learner.

### Table of contents
* [Preprocessing data and training the model](#training)
 * [Upload the data for training](#upload_data)
 * [Create a Scikit-learn script to train with](#create_sklearn_script)
 * [Create SageMaker Scikit Estimator](#create_sklearn_estimator)
 * [Batch transform our training data](#preprocess_train_data)
 * [Fit a LinearLearner Model with the preprocessed data](#training_model)
* [Inference Pipeline with Scikit preprocessor and Linear Learner](#inference_pipeline)
 * [Set up the inference pipeline](#pipeline_setup)
 * [Make a request to our pipeline endpoint](#pipeline_inference_request)
 * [Delete Endpoint](#delete_endpoint)

Let's first create our Sagemaker session and role, and create a S3 prefix to use for the notebook example.

In [10]:
import boto3, re, sys, math, json, os, sagemaker, urllib.request
from sagemaker import get_execution_role
import numpy as np                                
import pandas as pd                               
import matplotlib.pyplot as plt                   
from IPython.display import Image                 
from IPython.display import display               
from time import gmtime, strftime                 
from sagemaker.predictor import csv_serializer   
import io
import time
import json
import sagemaker.amazon.common as smac
%matplotlib inline

In [11]:
# S3 prefix
bucket_name = 'demo-saeed'
prefix = 'fraudcredit-pipeline'

import sagemaker
from sagemaker import get_execution_role

sagemaker_session = sagemaker.Session()

# Get a SageMaker-compatible role used by this Notebook Instance.
role = get_execution_role()

# Preprocessing data and training the model <a class="anchor" id="training"></a>


In [12]:
s3_input_train = sagemaker.s3_input(s3_data='s3://{}/{}/raw_train'.format(bucket_name, prefix))
s3_input_train.config

{'DataSource': {'S3DataSource': {'S3DataDistributionType': 'FullyReplicated',
   'S3DataType': 'S3Prefix',
   'S3Uri': 's3://demo-saeed/fraudcredit-pipeline/raw_train'}}}

## Create SageMaker Scikit Estimator <a class="anchor" id="create_sklearn_estimator"></a>

To run our Scikit-learn training script on SageMaker, we construct a `sagemaker.sklearn.estimator.sklearn` estimator, which accepts several constructor arguments:

* __entry_point__: The path to the Python script SageMaker runs for training and prediction.
* __role__: Role ARN
* __train_instance_type__ *(optional)*: The type of SageMaker instances for training. __Note__: Because Scikit-learn does not natively support GPU training, Sagemaker Scikit-learn does not currently support training on GPU instance types.
* __sagemaker_session__ *(optional)*: The session used to train on Sagemaker.

To see the code for the SKLearn Estimator, see here: https://github.com/aws/sagemaker-python-sdk/tree/master/src/sagemaker/sklearn

In [13]:
from sagemaker.sklearn.estimator import SKLearn

script_path = 'sklearn_fd_featurizer.py'

sklearn_preprocessor = SKLearn(
    entry_point=script_path,
    role=role,
    train_instance_type="ml.c4.xlarge",
    output_path='s3://{}/{}/output/'.format(bucket_name, prefix),
    #code_location='s3://<path-to-code_location>', read locally from notebook
    sagemaker_session=sagemaker_session)




In [14]:
sklearn_preprocessor.fit({'train': s3_input_train})

2019-05-30 20:07:22 Starting - Starting the training job...
2019-05-30 20:07:27 Starting - Launching requested ML instances......
2019-05-30 20:08:33 Starting - Preparing the instances for training...
2019-05-30 20:09:21 Downloading - Downloading input data...
2019-05-30 20:09:35 Training - Downloading the training image..
2019-05-30 20:09:58,894 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2019-05-30 20:09:58,897 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2019-05-30 20:09:58,908 sagemaker_sklearn_container.training INFO     Invoking user training script.
2019-05-30 20:09:59,128 sagemaker-containers INFO     Module sklearn_fd_featurizer does not provide a setup.py. 
Generating setup.py
2019-05-30 20:09:59,129 sagemaker-containers INFO     Generating setup.cfg
2019-05-30 20:09:59,129 sagemaker-containers INFO     Generating MANIFEST.in
2019-05-30 20:09:59,129 sagemaker-containers INFO     Installing module with t

In [15]:
sklearn_preprocessor.model_data

's3://demo-saeed/fraudcredit-pipeline/output/sagemaker-scikit-learn-2019-05-30-20-07-22-427/output/model.tar.gz'

In [16]:
sklearn_preprocessor.uploaded_code

UserCode(s3_prefix='s3://demo-saeed/sagemaker-scikit-learn-2019-05-30-20-07-22-427/source/sourcedir.tar.gz', script_name='sklearn_fd_featurizer.py')

## Batch transform our training data <a class="anchor" id="preprocess_train_data"></a>
Now that our proprocessor is properly fitted, let's go ahead and preprocess our training data. Let's use batch transform to directly preprocess the raw data and store right back into s3.

In [17]:
# Define a SKLearn Transformer from the trained SKLearn Estimator
transformer = sklearn_preprocessor.transformer(
    instance_count=1, 
    instance_type='ml.m4.xlarge',
    assemble_with = 'Line',
    output_path = 's3://{}/{}/preprocessed_model/'.format(bucket_name, prefix),
    accept = 'text/csv')

In [18]:
s3_input_train.config['DataSource']['S3DataSource']['S3Uri']

's3://demo-saeed/fraudcredit-pipeline/raw_train'

In [19]:
transformer.output_path

's3://demo-saeed/fraudcredit-pipeline/preprocessed_model/'

In [20]:
# Preprocess training input
transformer.output_path = 's3://demo-saeed/fraudcredit-pipeline/preprocessed_train/'
transformer.transform(s3_input_train.config['DataSource']['S3DataSource']['S3Uri'], content_type='text/csv', split_type='Line')
print('Waiting for transform job: ' + transformer.latest_transform_job.job_name)
transformer.wait()
preprocessed_train = transformer.output_path
s3_input_processed_train = sagemaker.session.s3_input(
    preprocessed_train, 
    distribution='FullyReplicated',
    content_type='text/csv', 
    s3_data_type='S3Prefix')
print(s3_input_processed_train.config)

Waiting for transform job: sagemaker-scikit-learn-2019-05-30-20-10-34-916
..........................................!
{'DataSource': {'S3DataSource': {'S3DataDistributionType': 'FullyReplicated', 'S3DataType': 'S3Prefix', 'S3Uri': 's3://demo-saeed/fraudcredit-pipeline/preprocessed_train/'}}, 'ContentType': 'text/csv'}


In [21]:
data_location = 's3://{}/{}/{}/{}'.format(bucket_name, prefix,'preprocessed_train','train.csv.out')
df = pd.read_csv(data_location,header = None)
df.head(25)

,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,30
0,1.0,0.005040,0.347824,-0.823091,0.214798,0.189050,0.550795,0.295456,0.775733,-0.092413,...,1.115517,0.018011,0.473297,1.113596,1.050489,-0.959868,-0.714505,-0.145987,0.197738,1.673615
1,0.0,-1.126314,0.652064,-0.477632,0.593031,-0.493070,0.361389,0.285038,0.505567,-0.107495,...,-0.040895,-0.241423,-0.427814,-0.127386,-0.522579,0.861587,0.616193,-0.143905,-0.047629,-0.253647
2,1.0,-0.163318,-0.374732,0.040842,0.173850,-0.857184,0.079320,-0.828474,0.093122,-0.011329,...,-0.833234,0.043010,-0.041022,-0.969610,0.614542,-0.084670,-2.135301,-1.114379,-0.075676,-0.180573
3,1.0,-1.577187,0.430722,0.678611,-0.528590,1.331444,0.696371,-1.060450,0.172643,-0.004949,...,0.443527,-0.021377,-0.476229,-0.583167,-1.201115,2.358365,1.110293,0.672299,0.985151,-0.460168
4,0.0,-0.935693,0.500394,-1.116306,0.499100,-0.735711,-0.025617,-0.033069,0.517261,-0.104463,...,0.230172,-0.193624,-0.803500,-0.348312,1.113977,0.138441,2.137330,-0.314881,0.072460,1.884638
5,0.0,1.185726,0.719633,-0.931972,0.606075,-0.479313,0.077990,1.303897,0.208337,0.057542,...,-0.032159,-0.082136,0.650188,-0.055082,0.387745,-0.326944,-0.456969,-0.047581,-0.094629,0.320423
6,0.0,0.916696,0.805108,-0.483740,0.240752,-0.595583,0.481676,-0.471306,0.633701,-0.197076,...,-0.218672,-0.104713,0.522820,-0.059814,0.341122,0.496978,1.718121,-0.249613,-0.246083,-0.277721
7,1.0,-1.193919,-3.385538,3.018868,-3.265232,1.266099,-3.453993,-2.141743,-2.440198,2.779663,...,1.425700,0.473639,-1.236831,-0.745679,0.361698,2.054090,-0.539148,1.575612,0.926249,0.008935
8,1.0,1.552768,-0.967128,-0.597244,-0.074438,0.092622,0.982852,1.162178,-0.761544,-3.273056,...,-0.595190,-2.438928,1.941238,1.448618,0.387589,0.189926,2.549625,1.246216,0.130140,-0.406571
9,0.0,1.424926,0.432989,-0.173383,0.471375,-0.763947,0.538402,-0.400965,0.665895,-0.132075,...,-0.128987,-0.056900,0.869997,-0.158206,-0.022869,-0.542452,-0.378043,0.151574,0.328365,-0.438985


In [ ]:
s3_input_validation = sagemaker.s3_input(s3_data='s3://{}/{}/raw_validation'.format(bucket_name, prefix))
s3_input_validation.config
# Preprocess validation input
transformer.output_path = 's3://demo-saeed/fraudcredit-pipeline/preprocessed_validation/'
transformer.transform(s3_input_validation.config['DataSource']['S3DataSource']['S3Uri'], content_type='text/csv', split_type='Line')
print('Waiting for transform job: ' + transformer.latest_transform_job.job_name)
transformer.wait()
preprocessed_validation = transformer.output_path
s3_input_processed_validation = sagemaker.session.s3_input(
    preprocessed_validation, 
    distribution='FullyReplicated',
    content_type='text/csv', 
    s3_data_type='S3Prefix')
print(s3_input_processed_validation.config)

In [ ]:
s3_input_test = sagemaker.s3_input(s3_data='s3://{}/{}/raw_test'.format(bucket_name, prefix))
s3_input_test.config
# Preprocess training input
transformer.output_path = 's3://demo-saeed/fraudcredit-pipeline/preprocessed_test/'
transformer.transform(s3_input_test.config['DataSource']['S3DataSource']['S3Uri'], content_type='text/csv', split_type='Line')
print('Waiting for transform job: ' + transformer.latest_transform_job.job_name)
transformer.wait()
preprocessed_test = transformer.output_path
s3_input_processed_test = sagemaker.session.s3_input(
    preprocessed_test, 
    distribution='FullyReplicated',
    content_type='text/csv', 
    s3_data_type='S3Prefix')
print(s3_input_processed_test.config)

## Fit a LinearLearner Model with the preprocessed data <a class="anchor" id="training_model"></a>
Let's take the preprocessed training data and fit a LinearLearner Model. Sagemaker provides prebuilt algorithm containers that can be used with the Python SDK. The previous Scikit-learn job preprocessed the raw Titanic dataset into labeled, useable data that we can now use to fit a binary classifier Linear Learner model.

For more on Linear Learner see: https://docs.aws.amazon.com/sagemaker/latest/dg/linear-learner.html

# training

In [22]:
from sagemaker.amazon.amazon_estimator import get_image_uri
container = get_image_uri(boto3.Session().region_name, 'linear-learner')
container

'382416733822.dkr.ecr.us-east-1.amazonaws.com/linear-learner:1'

In [23]:
output_location = 's3://{}/{}/model_output'.format(bucket_name, prefix)
print('training artifacts will be uploaded to: {}'.format(output_location))

training artifacts will be uploaded to: s3://demo-saeed/fraudcredit-pipeline/model_output


In [24]:
s3_input_processed_train.config

{'DataSource': {'S3DataSource': {'S3DataDistributionType': 'FullyReplicated',
   'S3DataType': 'S3Prefix',
   'S3Uri': 's3://demo-saeed/fraudcredit-pipeline/preprocessed_train/'}},
 'ContentType': 'text/csv'}

In [ ]:
import pandas as pd
data_location = 's3://{}/{}/raw_train/train.csv'.format(bucket_name, prefix)

data = pd.read_csv(data_location, header=None)
data.head()

In [25]:
import pandas as pd
data_location = 's3://{}/{}/preprocessed_train/train.csv.out'.format(bucket_name, prefix)

data = pd.read_csv(data_location, header=None)
data.head()

,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,30
0,1.0,0.005040,0.347824,-0.823091,0.214798,0.189050,0.550795,0.295456,0.775733,-0.092413,...,1.115517,0.018011,0.473297,1.113596,1.050489,-0.959868,-0.714505,-0.145987,0.197738,1.673615
1,0.0,-1.126314,0.652064,-0.477632,0.593031,-0.493070,0.361389,0.285038,0.505567,-0.107495,...,-0.040895,-0.241423,-0.427814,-0.127386,-0.522579,0.861587,0.616193,-0.143905,-0.047629,-0.253647
2,1.0,-0.163318,-0.374732,0.040842,0.173850,-0.857184,0.079320,-0.828474,0.093122,-0.011329,...,-0.833234,0.043010,-0.041022,-0.969610,0.614542,-0.084670,-2.135301,-1.114379,-0.075676,-0.180573
3,1.0,-1.577187,0.430722,0.678611,-0.528590,1.331444,0.696371,-1.060450,0.172643,-0.004949,...,0.443527,-0.021377,-0.476229,-0.583167,-1.201115,2.358365,1.110293,0.672299,0.985151,-0.460168
4,0.0,-0.935693,0.500394,-1.116306,0.499100,-0.735711,-0.025617,-0.033069,0.517261,-0.104463,...,0.230172,-0.193624,-0.803500,-0.348312,1.113977,0.138441,2.137330,-0.314881,0.072460,1.884638


In [32]:
output_location

's3://demo-saeed/fraudcredit-pipeline/model_output'

In [27]:
import boto3
import sagemaker

sess = sagemaker.Session()

linear = sagemaker.estimator.Estimator(container,
                                       role, 
                                       train_instance_count=1, 
                                       train_instance_type='ml.m4.2xlarge',
                                       output_path=output_location,
                                       sagemaker_session=sess)
linear.set_hyperparameters(feature_dim=30,
                           predictor_type='binary_classifier',
                           epochs = 1,
                           mini_batch_size=200)
linear.fit({'train': s3_input_processed_train,  'validation': s3_input_processed_validation, 'test': s3_input_processed_test})
#linear.fit({'train': s3_input_processed_train})

# train_max_run = 3600,

2019-05-30 20:16:20 Starting - Starting the training job...
2019-05-30 20:16:21 Starting - Launching requested ML instances.........
2019-05-30 20:17:55 Starting - Preparing the instances for training...
2019-05-30 20:18:42 Downloading - Downloading input data
2019-05-30 20:18:42 Training - Downloading the training image.....
Docker entrypoint called with argument(s): train
[05/30/2019 20:19:26 INFO 139819483055936] Reading default configuration from /opt/amazon/lib/python2.7/site-packages/algorithm/default-input.json: {u'loss_insensitivity': u'0.01', u'epochs': u'15', u'init_bias': u'0.0', u'lr_scheduler_factor': u'auto', u'num_calibration_samples': u'10000000', u'accuracy_top_k': u'3', u'_num_kv_servers': u'auto', u'use_bias': u'true', u'num_point_for_scaler': u'10000', u'_log_level': u'info', u'quantile': u'0.5', u'bias_lr_mult': u'auto', u'lr_scheduler_step': u'auto', u'init_method': u'uniform', u'init_sigma': u'0.01', u'lr_scheduler_minimum_lr': u'auto', u'target_recall': u'0.8', 

In [33]:
linear.output_path

's3://demo-saeed/fraudcredit-pipeline/model_output'

In [29]:
%matplotlib inline
from sagemaker.analytics import TrainingJobAnalytics

training_job_name = linear._current_job_name
metric_name = 'validation:binary_f_beta'

metrics_dataframe = TrainingJobAnalytics(training_job_name=training_job_name,metric_names=[metric_name]).dataframe()
plt = metrics_dataframe.plot(kind='line', figsize=(12,5), x='timestamp', y='value', style='b.', legend=False)
plt.set_ylabel(metric_name);

KeyError: 'timestamp'

In [39]:
#linear.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')
linear_predictor = linear.deploy(initial_instance_count=1, instance_type='ml.c5.xlarge', endpoint_name='pip-model-aws-linear-learner1', update_endpoint=True)


---------------------------------------------------------------------------------------------------!

In [40]:
linear.output_path

's3://demo-saeed/fraudcredit-pipeline/model_output'

In [41]:
linear.latest_training_job.job_name

'linear-learner-2019-05-30-20-16-20-007'

In [42]:
linear_predictor.content_type

In [43]:
linear_predictor.endpoint

'pip-model-aws-linear-learner1'

In [45]:
data_location = 's3://{}/{}/{}/{}'.format(bucket_name, prefix,'preprocessed_train','train.csv.out')
df = pd.read_csv(data_location,header = None)
df.head(5)

,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,30
0,1.0,0.005040,0.347824,-0.823091,0.214798,0.189050,0.550795,0.295456,0.775733,-0.092413,...,1.115517,0.018011,0.473297,1.113596,1.050489,-0.959868,-0.714505,-0.145987,0.197738,1.673615
1,0.0,-1.126314,0.652064,-0.477632,0.593031,-0.493070,0.361389,0.285038,0.505567,-0.107495,...,-0.040895,-0.241423,-0.427814,-0.127386,-0.522579,0.861587,0.616193,-0.143905,-0.047629,-0.253647
2,1.0,-0.163318,-0.374732,0.040842,0.173850,-0.857184,0.079320,-0.828474,0.093122,-0.011329,...,-0.833234,0.043010,-0.041022,-0.969610,0.614542,-0.084670,-2.135301,-1.114379,-0.075676,-0.180573
3,1.0,-1.577187,0.430722,0.678611,-0.528590,1.331444,0.696371,-1.060450,0.172643,-0.004949,...,0.443527,-0.021377,-0.476229,-0.583167,-1.201115,2.358365,1.110293,0.672299,0.985151,-0.460168
4,0.0,-0.935693,0.500394,-1.116306,0.499100,-0.735711,-0.025617,-0.033069,0.517261,-0.104463,...,0.230172,-0.193624,-0.803500,-0.348312,1.113977,0.138441,2.137330,-0.314881,0.072460,1.884638


In [87]:
X_test = df.iloc[0:1, 1:]
# X_test.columns =['Time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10',
#        'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20',
#        'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount']
X_test

,1,2,3,4,5,6,7,8,9,10,...,21,22,23,24,25,26,27,28,29,30
0,0.00504,0.347824,-0.823091,0.214798,0.18905,0.550795,0.295456,0.775733,-0.092413,0.156344,...,1.115517,0.018011,0.473297,1.113596,1.050489,-0.959868,-0.714505,-0.145987,0.197738,1.673615


In [83]:
from sagemaker.predictor import csv_serializer, json_deserializer

linear_predictor.content_type = 'text/csv'
linear_predictor.serializer = csv_serializer
linear_predictor.deserializer = json_deserializer
#linear_predictor.predict(X_test.iloc[0])

linear_predictor.predict(X_test.values)


{'predictions': [{'score': 0.258267343044281, 'predicted_label': 1.0}]}

# Serial Inference Pipeline with Scikit preprocessor and Linear Learner <a class="anchor" id="serial_inference"></a>


## Set up the inference pipeline <a class="anchor" id="pipeline_setup"></a>
Setting up a Machine Learning pipeline can be done with the Pipeline Model. This sets up a list of models in a single endpoint; in this example, we configure our pipeline model with the fitted Scikit-learn inference model and the fitted Linear Learner model. Deploying the model follows the same ```deploy``` pattern in the SDK.

In [84]:
from sagemaker.model import Model
from sagemaker.pipeline import PipelineModel
import boto3
from time import gmtime, strftime

timestamp_prefix = strftime("%Y-%m-%d-%H-%M-%S", gmtime())

scikit_learn_inferencee_model = sklearn_preprocessor.create_model()
linear_learner_model = linear.create_model()

model_name = 'inference-pipeline-' + timestamp_prefix
endpoint_name = 'inference-pipeline-ep-' + timestamp_prefix
sm_model = PipelineModel(
    name=model_name, 
    role=role, 
    models=[
        scikit_learn_inferencee_model, 
        linear_learner_model])

sm_model.deploy(initial_instance_count=1, instance_type='ml.c4.xlarge', endpoint_name=endpoint_name)

----------------------------------------------------------------------------------------------------!

In [85]:
sm_model.endpoint_name

'inference-pipeline-ep-2019-05-30-21-34-46'

## Make a request to our pipeline endpoint <a class="anchor" id="pipeline_inference_request"></a>

Here we just grab the first line from the test data (you'll notice that the inference python script is very particular about the ordering of the inference request data). The ```ContentType``` field configures the first container, while the ```Accept``` field configures the last container. You can also specify each container's ```Accept``` and ```ContentType``` values using environment variables.

We make our request with the payload in ```'text/csv'``` format, since that is what our script currently supports. If other formats need to be supported, this would have to be added to the ```output_fn()``` method in our entry point. Note that we set the ```Accept``` to ```application/json```, since Linear Learner does not support ```text/csv``` ```Accept```. The prediction output in this case is trying to guess the number of rings the abalone specimen would have given its other physical features; the actual number of rings is 10.

In [89]:
X_test

,1,2,3,4,5,6,7,8,9,10,...,21,22,23,24,25,26,27,28,29,30
0,0.00504,0.347824,-0.823091,0.214798,0.18905,0.550795,0.295456,0.775733,-0.092413,0.156344,...,1.115517,0.018011,0.473297,1.113596,1.050489,-0.959868,-0.714505,-0.145987,0.197738,1.673615


In [92]:
from sagemaker.predictor import json_serializer, csv_serializer, json_deserializer, RealTimePredictor
from sagemaker.content_types import CONTENT_TYPE_CSV, CONTENT_TYPE_JSON
payload = X_test.values
actual_rings = 10
predictor = RealTimePredictor(
    endpoint=endpoint_name,
    sagemaker_session=sagemaker_session,
    serializer=csv_serializer,
    content_type=CONTENT_TYPE_CSV,
    accept=CONTENT_TYPE_JSON)

print(predictor.predict(payload))


b'{"predictions": [{"score": 0.07547429949045181, "predicted_label": 0.0}]}'


## Delete Endpoint <a class="anchor" id="delete_endpoint"></a>
Once we are finished with the endpoint, we clean up the resources!

In [ ]:
# sm_client = sagemaker_session.boto_session.client('sagemaker')
# sm_client.delete_endpoint(EndpointName=endpoint_name)